In [17]:
# Importing the StringIO module.
from bs4 import BeautifulSoup

# import the requests library 
import requests

import pandas as pd

#Base Url to fetch historical data through request
#List holds various stocks
#lst = ['MRF','SBIN','TCS','HCLTECH','ROSSARI','PIDILITIND']


#Defining headers with User agent to establish requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest'
}

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import csv

import pandas as pd


In [27]:
options = Options()
options.add_argument('--headless')

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

#driver = webdriver.Firefox(options=options)
driver.get("https://marketsmithindia.com/mstool/eval/ROSSARI/evaluation.jsp#/")
#time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')


In [28]:
#Populating summary columns for individual stocks from Marketsmith evaluation pages
stock_detail_header = soup.find_all('div', class_ = {"details"})
stock_detail_header

[<div class="details"> Market Capitalization</div>,
 <div class="details"> Sales </div>,
 <div class="details"> Shares in Float </div>,
 <div class="details"> No of Funds </div>,
 <div class="details"> Shares held by Funds </div>,
 <div class="details"> Market Capitalization </div>,
 <div class="details"> Sales </div>,
 <div class="details"> Shares in Float </div>,
 <div class="details"> No of Funds </div>,
 <div class="details"> Shares held by Funds </div>,
 <div class="details"> Yield </div>,
 <div class="details"> Book Value </div>,
 <div class="details"> U/D Vol Ratio </div>,
 <div class="details"> LTDebt/Equity </div>,
 <div class="details"> Alpha </div>,
 <div class="details"> Beta </div>,
 <div class="details"> Master Score </div>,
 <div class="details"> EPS Rating </div>,
 <div class="details"> Price Strength </div>,
 <div class="details"> Acc/Dis Rating </div>,
 <div class="details"> Group Rank </div>,
 <div class="details"> EPS Growth Rate </div>,
 <div class="details"> Earni

In [10]:
#Populating stock summary values for respective for individual stocks from Marketsmith evaluation pages

stock_detail_value = soup.find_all('div', class_ = {"value"})
stock_detail_value


[<div class="value" style="font-weight: bold;">INR 4,309.19 Cr</div>,
 <div class="value" style="font-weight: bold;">INR 1152.75 Cr</div>,
 <div class="value" style="font-weight: bold;">1.77 Cr</div>,
 <div class="value" style="font-weight: bold;">69 <span class="nooffundspval">(+2.99 %)</span></div>,
 <div class="value" style="font-weight: bold;">1.20 Cr <span class="nooffundsnval">(-1.92 %)</span></div>,
 <div class="value" style="font-weight: bold;">INR 4,309.19 Cr</div>,
 <div class="value" style="font-weight: bold;">INR 1152.75 Cr</div>,
 <div class="value" style="font-weight: bold;">1.77 Cr</div>,
 <div class="value" style="font-weight: bold;">69 <span class="nooffundspval">(+2.99 %)</span></div>,
 <div class="value" style="font-weight: bold;">1.20 Cr <span class="nooffundsnval">(-1.92 %)</span></div>,
 <div class="value">0.06%</div>,
 <div class="value">5.01x</div>,
 <div class="value">1.2</div>,
 <div class="value">4%</div>,
 <div class="value">-0.04</div>,
 <div class="value">

In [29]:
cols = []

#Generating columns through iteration
for x in range(0,len(stock_detail_header)):
    cols.append(stock_detail_header[x].get_text().strip())
    
cols

['Market Capitalization',
 'Sales',
 'Shares in Float',
 'No of Funds',
 'Shares held by Funds',
 'Market Capitalization',
 'Sales',
 'Shares in Float',
 'No of Funds',
 'Shares held by Funds',
 'Yield',
 'Book Value',
 'U/D Vol Ratio',
 'LTDebt/Equity',
 'Alpha',
 'Beta',
 'Master Score',
 'EPS Rating',
 'Price Strength',
 'Acc/Dis Rating',
 'Group Rank',
 'EPS Growth Rate',
 'Earnings Stability',
 'P/E Ratio',
 '5-Year P/E Range',
 'Return on Equity',
 'Cash Flow (INR)']

In [30]:
#Feeding an empty Dataframe with its respective column values through iterating summary_data list
#Creating an empty Dataframe to feed the column names through iterating summary_cols list

summary_data_values = []

#Generating columns through iteration
for x in range(0,len(stock_detail_value)):
    print(stock_detail_value[x].get_text().strip())
    summary_data_values.append(stock_detail_value[x].get_text().strip())

INR 4,309.19 Cr
INR 1152.75 Cr
1.77 Cr
69 (+2.99 %)
1.20 Cr (-1.92 %)
INR 4,309.19 Cr
INR 1152.75 Cr
1.77 Cr
69 (+2.99 %)
1.20 Cr (-1.92 %)
0.06%
5.01x
1.2
4%
-0.04
0.83
57
84
28
B
31 of 197
N/A
61
45
47-
                                    104
8%
20.61


In [31]:
#Creating an empty Dataframe to feeding the Stock summary column names and values together from Marketsmith website
share_summary = pd.DataFrame(columns= cols,data = [summary_data_values])



In [32]:
share_summary

,Market Capitalization,Sales,Shares in Float,No of Funds,Shares held by Funds,Market Capitalization,Sales,Shares in Float,No of Funds,Shares held by Funds,...,EPS Rating,Price Strength,Acc/Dis Rating,Group Rank,EPS Growth Rate,Earnings Stability,P/E Ratio,5-Year P/E Range,Return on Equity,Cash Flow (INR)
0,"INR 4,309.19 Cr",INR 1152.75 Cr,1.77 Cr,69 (+2.99 %),1.20 Cr (-1.92 %),"INR 4,309.19 Cr",INR 1152.75 Cr,1.77 Cr,69 (+2.99 %),1.20 Cr (-1.92 %),...,84,28,B,31 of 197,N/A,61,45,47-\n 104,8%,20.61


In [25]:
del[share_summary]